In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import Elastic_Functions as elfunc
import General_Functions as genfunc
import QMatrix
import sys
import time
np.set_printoptions(threshold=sys.maxsize)


# MATERIAL PARAMETERS
M_s = 1.5e6
gyromagnetic_ratio = 1.761e11
exchange_length = 3e-9
lambda_m: float = 30e-6  # saturation magnetostrain parameter
mu_0 = 1.25663706e-6
density = 7874
KAPPA: float = genfunc.calculate_KAPPA(density, exchange_length, gyromagnetic_ratio, mu_0)  # Determines the relative strength of the elastic vs. magnetic parts.
stress_fac = genfunc.stress_density_factor(KAPPA=KAPPA, mu_0=mu_0, M_s=M_s)
mu, lam = 54e9/stress_fac, 172e9/stress_fac
ALPHA: float = 1  # Dissipative constant in the LLG equation.

# SIMULATION PARAMETERS
T_MAX_DIM: float = 1e-10  # The maximum time for the simulation in seconds
THETA: float = 0.505  # Should be strictly above 1/2 for unconditional stability
time_step_dim = 1e-12 # seconds
K: float = time_step_dim*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)  # TIME STEP
T_MAX: float = T_MAX_DIM*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)
H_MAX: float = 3  # Determines how fine the mesh should be.

print(f"µ = {mu}, λ = {lam}, κ = {KAPPA}")
print(f"K = {K}, T_MAX = {T_MAX}")

In [ ]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(6, 6, 6), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(6, 6, 6), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(6, 6, 6), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [ ]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="left"
)  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
prev_disp_gfu = GridFunction(fes_disp) #  used to store the previous displacement
zeeman_factor = Parameter(0)
f_zeeman = CoefficientFunction((0, 0, zeeman_factor))
# body force and traction force
body_factor = Parameter(genfunc.force_density_grav(grav_accel=-9.81, density=7874, exchange_length=3e-9, KAPPA=KAPPA, mu_0=1.25663706e-6, M_s=217.6))
body_factor = 0.0
f_body = CoefficientFunction((0.0, 0.0, body_factor))
surface_factor = Parameter(0)
g_surface = CoefficientFunction([(0,surface_factor,0) if bc=="top" else (0,0,0) for bc in mesh.GetBoundaries()])

In [ ]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
#disp_gfu = elfunc.give_random_displacement(disp_gfu)
disp_gfu = elfunc.give_uniform_displacement(disp_gfu, (0.0, 0.0, 0.0))  # initial displacement
velocity_gfu = GridFunction(fes_disp)  # velocity
velocity_gfu = elfunc.give_uniform_displacement(velocity_gfu, (0.0, 0.0, 0.0))  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu, mesh, "magnetisation")
Draw(disp_gfu, mesh, "displacement")
proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)

a_mag_fixed = magfunc.build_fixed_mag(fes_mag, ALPHA, THETA, K)  # static portion of the magnetisation linear system

In [ ]:
display(np.ones(2*5))
display(QMatrix.q_block(np.ones(2*5)).todense())